In [16]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_parquet('/opt/data/california-worldcover-chips-osm-multilabels.parquet')
df.shape

(83609, 8)

In [3]:
pd.DataFrame.query?

Signature: pd.DataFrame.query(self, expr: 'str', *, inplace: 'bool' = False, **kwargs) -> 'DataFrame | None'
Docstring:
Query the columns of a DataFrame with a boolean expression.

Parameters
----------
expr : str
    The query string to evaluate.

    You can refer to variables
    in the environment by prefixing them with an '@' character like
    ``@a + b``.

    You can refer to column names that are not valid Python variable names
    by surrounding them in backticks. Thus, column names containing spaces
    or punctuations (besides underscores) or starting with digits must be
    surrounded by backticks. (For example, a column named "Area (cm^2)" would
    be referenced as ```Area (cm^2)```). Column names which are Python keywords
    (like "list", "for", "import", etc) cannot be used.

    For example, if one of your columns is called ``a a`` and you want
    to sum it with ``b``, your query should be ```a a` + b``.

inplace : bool
    Whether to modify the DataFrame rather than

In [17]:
df.head()

,onehot_count,onehot_area,onehot_length,string_labels,col,row,geometry,split
chip_id,,,,,,,,
193c3415fc237,"[24, 24, 1344, 388, 472, 21, 453, 2, 8, 0, 0, ...","[58033.967200952815, 58033.967200952815, 23937...","[4913.205774298301, 4913.205774298301, 74943.4...","[amenity=*, amenity=parking, building=*, build...",78080,96000,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
1639b12583426,"[148, 148, 4171, 1245, 1891, 52, 938, 13, 32, ...","[160073.78667008967, 160073.78667008967, 80191...","[20966.344659970102, 20966.344659970102, 24881...","[amenity=*, amenity=parking, building=*, build...",78080,95744,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
253904d963d6a,"[65, 65, 200, 104, 76, 0, 18, 2, 0, 0, 0, 0, 0...","[101980.95118259436, 101980.95118259436, 71246...","[10597.989172272657, 10597.989172272657, 16664...","[amenity=*, amenity=parking, building=*, build...",77824,95744,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
00530dca88d4a,"[9, 9, 4897, 688, 4026, 0, 172, 10, 1, 0, 0, 0...","[17637.908172952746, 17637.908172952746, 10930...","[1925.4953839123748, 1925.4953839123748, 32065...","[amenity=*, amenity=parking, building=*, build...",77824,95488,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train
0e554bbdbe661,"[80, 80, 4358, 1262, 2386, 59, 597, 27, 27, 0,...","[131950.60085068244, 131950.60085068244, 98472...","[13536.03447682216, 13536.03447682216, 283601....","[amenity=*, amenity=parking, building=*, build...",78080,95488,b'\x01\x03\x00\x00\x00\x01\x00\x00\x00\x05\x00...,train


In [28]:
mat_OSM_counts = np.stack(df.onehot_count)
print(mat_OSM_counts.shape)
print(mat_OSM_counts)

(83609, 99)
[[  24   24 1344 ...   53    0    0]
 [ 148  148 4171 ...  166    0    1]
 [  65   65  200 ...   29    0    1]
 ...
 [   4    4    1 ...   77    0   40]
 [   0    0    0 ...    4    0   12]
 [   0    0    0 ...    0    0    7]]


TODO
- assume `autocomplete_query` accepts a constraint matrix instead of a dataframe  

In [5]:
def autocomplete_query(
    constraints: tuple[int],  # user contraints, for ex. (0,0,...,1,-1)
    df: pd.DataFrame,
    n_samples: int = 1,
):
    constraints_mask = (constraints != 0)  # mask of constrained columns
    constrainted_cols = df.columns[constraints_mask]
    df = df.copy()
    
    for i, col in enumerate(constrainted_cols):
        if constraints[i] == 1:
            df = df.query('@col == 1')
            # df = df[df[col] == 1]
        elif constraints[i] == -1:
            df = df.query('@col == 0')
            # df = df[df[col] == 0]

    # sample w replacement
    df_sample = df.sample(n=n_samples, replace=True, random_state=42, axis=0)

    return df_sample

In [31]:
mat_OSM_counts[0]

array([  24,   24, 1344,  388,  472,   21,  453,    2,    8,    0,    0,
          0,    0,   53,    0,   36,    0,    0,    0,    0,    0,    0,
          0,   31,    0,    0,    0,    0,    0,    0,    0,    5,    0,
         53,    5,    0,    0,    5,   41,    1,    0,    0,    1,    3,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    3,
          0,    0,    0,    0,    0,    0,   11,    0,   11,    0,    0,
          0,    0,    1,    0,    0,    0,    0,    0,    0,    1,    0,
         18,    4,    1,    0,    0,    4,    9,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,   53,    0,    0],
      dtype=int32)

In [45]:
example_constraints = np.array((0,) * 99)
example_constraints[0] = 1
example_constraints[1] = -1
example_constraints

array([ 1, -1,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
        0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0])